# Input multiomic dataset
The data used in this study was downloaded from the preloaded gene expression, methylation, and proteomics datasets for the TCGA breast invasive carcinoma (BRCA) dataset from https://software.broadinstitute.org/morpheus/ (July 2020)

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc

def get_predictions(predictions_proba, threshold=0.5):
  predictions = np.where(predictions_proba <= threshold, 0, 1)
  return predictions

def buildTrainingDataset(geneList):
    metaData = pd.read_csv("breastCancer_metaData.csv",index_col=0)
    severityLabel = metaData['Severity']
    
    mrnaData = pd.read_csv("breastCancer_mrnaData.csv",index_col=0)
    mrnaData = mrnaData[set(geneList) & set(mrnaData.columns)]
    
    methData = pd.read_csv("breastCancer_methData.csv",index_col=0)
    methData = methData[set(geneList) & set(methData.columns)]
    
    protData = pd.read_csv("breastCancer_protData.csv",index_col=0)
    protData = protData[set(geneList) & set(protData.columns)]

    #fill na's with mean for group
    mrnaData_Severe = mrnaData.loc[severityLabel.index[severityLabel == 1]]
    mrnaData_Severe = mrnaData_Severe.fillna(mrnaData_Severe.mean())
    mrnaData_NotSevere = mrnaData.loc[severityLabel.index[severityLabel == 0]]
    mrnaData_NotSevere = mrnaData_NotSevere.fillna(mrnaData_NotSevere.mean())
    mrnaData = pd.concat([mrnaData_Severe, mrnaData_NotSevere])
    mrnaData = mrnaData.loc[severityLabel.index]
    
    methData_Severe = methData.loc[severityLabel.index[severityLabel == 1]]
    methData_Severe = methData_Severe.fillna(methData_Severe.mean())
    methData_NotSevere = methData.loc[severityLabel.index[severityLabel == 0]]
    methData_NotSevere = methData_NotSevere.fillna(methData_NotSevere.mean())
    methData = pd.concat([methData_Severe, methData_NotSevere])
    methData = methData.loc[severityLabel.index]
    
    protData_Severe = protData.loc[severityLabel.index[severityLabel == 1]]
    protData_Severe = protData_Severe.fillna(protData_Severe.mean())
    protData_NotSevere = protData.loc[severityLabel.index[severityLabel == 0]]
    protData_NotSevere = protData_NotSevere.fillna(protData_NotSevere.mean())
    protData = pd.concat([protData_Severe, protData_NotSevere])
    protData = protData.loc[severityLabel.index]
    
    multiData = pd.merge(pd.merge(mrnaData,methData, left_index=True, right_index=True),
                         protData,left_index=True, right_index=True)
    
    return severityLabel, mrnaData, methData, protData, multiData

def train_models(X,Y,listName,datasetName):
    #Scaling the dataframe
    scaler = MinMaxScaler(feature_range=(0,1), copy=True)
    X = scaler.fit_transform(X)
    # prepare models
    models = []
    models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=1000)))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('RF', RandomForestClassifier()))
    models.append(('NB', GaussianNB()))

    # Evaluate each model
    model_results = pd.DataFrame({'Severity' : Y})
    rocPlots = pd.DataFrame(columns=['Dataset','Gene List','Model','FPR','TPR','Thresholds'])
    for name, model in models:    
        kfold = model_selection.StratifiedKFold(n_splits=10)
        predictions_proba = model_selection.cross_val_predict(model, X, Y, cv=kfold, method='predict_proba')
        fpr, tpr, thresholds = roc_curve(Y,predictions_proba[:,1])
        rocPlots_model = pd.DataFrame({'Dataset' : [datasetName]*len(fpr),
                                       'Gene List' : [listName]*len(fpr),
                                       'Model' : [name]*len(fpr),
                                       'FPR' : fpr,
                                       'TPR' : tpr,
                                       'Thresholds' : thresholds})
       
        rocPlots = rocPlots.append(rocPlots_model,sort=False)
        model_results[name] = predictions_proba[:,1]
    
    
    return model_results, rocPlots



## Preprocess the data
The dataset is transformed to create our model dataset by selecting for patients that fit our labeling criteria. It inputs the Gene Cluster Text (gct) format (https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats#GCT:_Gene_Cluster_Text_file_format_.28.2A.gct.29)

In [34]:
#inputing and formatting the data frame
data = pd.read_csv("breast_cancer_MorpheusTCGA.gtc",sep='\t',skiprows=2,low_memory=False)
data = data.drop(data.index[0])
datareset = data
data

#parse the metadata
metaData = data[data['Source'] == 'na']
metaData = metaData.T
metaData.columns = metaData.iloc[0]
metaData = metaData.drop(metaData.index[ [0,1,2] ])

#parse the mrna Data
mrnaData = data[data['Source'] == 'mRNAseq_RSEM_normalized_log2']
mrnaData = mrnaData.T
mrnaData.columns = mrnaData.iloc[0]
mrnaData = mrnaData.drop(mrnaData.index[ [0,1,2] ])

#parse out methylation data
methData = data[data['Source'] == 'meth.by_mean.data']
methData = methData.T
methData.columns = methData.iloc[0]
methData = methData.drop(methData.index[ [0,1,2] ])

#parse out proteomics data
protData = data[data['Source'] == 'rppa']
protData = protData.T
protData.columns = protData.iloc[0]
protData = protData.drop(protData.index[ [0,1,2] ])

#inclusion criteria
cancerTypeFilter = ((metaData['sample_type'] == 'Primary solid Tumor') & 
                    (metaData['histological_type'] == 'infiltrating ductal carcinoma'))

severeFilter = metaData['days_to_death'].fillna(36500).astype(int) < 1827
numSevere = len(metaData[cancerTypeFilter & severeFilter])
followupThreshold = metaData[cancerTypeFilter]['days_to_last_followup'].fillna(0).astype(int).sort_values()[-numSevere]
followupFilter = metaData['days_to_last_followup'].fillna(0).astype(int) >= followupThreshold

#Filtered Population
metaData = metaData[(cancerTypeFilter & (severeFilter | followupFilter))]
mrnaData = mrnaData[(cancerTypeFilter & (severeFilter | followupFilter))].astype(float)
methData = methData[(cancerTypeFilter & (severeFilter | followupFilter))].astype(float)
protData = protData[(cancerTypeFilter & (severeFilter | followupFilter))].astype(float)

#applying label to datasets (severe population = 1)
metaData.insert(0,"Severity",severeFilter.astype(int))
mrnaData.insert(0,"Severity",severeFilter.astype(int))
methData.insert(0,"Severity",severeFilter.astype(int))
protData.insert(0,"Severity",severeFilter.astype(int))

#print the processed data files
metaData.to_csv("breastCancer_metaData.csv",na_rep='NA')
mrnaData.to_csv("breastCancer_mrnaData.csv",na_rep='NA')
methData.to_csv("breastCancer_methData.csv",na_rep='NA')
protData.to_csv("breastCancer_protData.csv",na_rep='NA')

In [3]:
data = pd.read_csv("breast_cancer_MorpheusTCGA.gtc",sep='\t',skiprows=2,low_memory=False)
data = data.drop(data.index[0])
datareset = data
data

,id,Source,TCGA-3C-AALI-01A-21-A43F-20,aaau-Primary solid Tumor,aali-Primary solid Tumor,aalj-Primary solid Tumor,aalk-Primary solid Tumor,aaak-Primary solid Tumor,aat0-Primary solid Tumor,aat1-Primary solid Tumor,...,aaz6-Primary solid Tumor,a93s-Primary solid Tumor,a7hq-Primary solid Tumor,a86g-Primary solid Tumor,ab41-Primary solid Tumor,ab44-Primary solid Tumor,a899-Primary solid Tumor,a89a-Primary solid Tumor,a8r5-Primary solid Tumor,a8r6-Primary solid Tumor
1,sample_type,na,na,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,...,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor,Primary solid Tumor
2,mRNAseq_cluster,na,na,1,2,1,3,3,3,4,...,1,2,4,3,1,2,6,6,3,1
3,bcr_patient_barcode,na,na,tcga-3c-aaau,tcga-3c-aali,tcga-3c-aalj,tcga-3c-aalk,tcga-4h-aaak,tcga-5l-aat0,tcga-5l-aat1,...,tcga-ul-aaz6,tcga-uu-a93s,tcga-v7-a7hq,tcga-w8-a86g,tcga-wt-ab41,tcga-wt-ab44,tcga-xx-a899,tcga-xx-a89a,tcga-z7-a8r5,tcga-z7-a8r6
4,bcr_patient_uuid,na,na,6e7d5ec6-a469-467c-b748-237353c23416,55262fcb-1b01-4480-b322-36570430c917,427d0648-3f77-4ffc-b52c-89855426d647,c31900a4-5dcd-4022-97ac-638e86e889e4,6623fc5e-00be-4476-967a-cbd55f676ea6,86c6f993-327f-4525-9983-29c55625593a,16fc3677-0393-4ed1-ad3f-c8355f056369,...,4b54e06e-a280-4981-a4e1-9aea154341b4,45013972-2dfd-4f82-a076-e3e4af1b43b8,1285eb55-415c-494a-aa58-936f0427cdd0,90ff48c3-b14b-4a8b-94a1-98c0bab0d27b,e7db08a7-b439-4230-8dc4-1b54af4736c4,5cd79093-1571-4f71-8136-0d84ccabdcac,f89588e9-ca73-4465-a7fb-7246edb45e3a,ca20249f-b7ea-4fd9-9ecb-34f74755ae35,23f438bd-1dbb-4d46-972f-1e8e74ddbd37,b1d44c81-747d-471f-9093-aeb262a17975
5,vital_status,na,na,alive,alive,alive,alive,alive,alive,alive,...,alive,dead,alive,alive,alive,alive,alive,alive,alive,alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38762,ZYX,meth.by_mean.data,undefined,0.22,0.24,0.21,0.22,0.23,0.21,0.22,...,0.26,0.16,0.22,0.22,0.23,0.24,0.21,0.18,0.23,0.21
38763,ZZEF1,meth.by_mean.data,undefined,0.48,0.46,0.48,0.49,0.49,0.48,0.48,...,0.47,0.47,0.48,0.48,0.48,0.5,0.48,0.48,0.47,0.5
38764,ZZZ3,meth.by_mean.data,undefined,0.11,0.12,0.1,0.11,0.12,0.11,0.11,...,0.12,0.12,0.09,0.1,0.11,0.13,0.1,0.1,0.11,0.1
38765,psiTPTE22,meth.by_mean.data,undefined,0.25,0.16,0.25,0.47,0.31,0.36,0.4,...,0.33,0.49,0.2,0.17,0.35,0.14,0.18,0.17,0.26,0.32


## Generate and Evaluate Classification Models
First the genes of interest are extracted from the input data to build a test and training set.

The first set of unsupervised methods extracts the top 20 most variable genes from the dataset.

In [ ]:
### This reruns the unsupervised analysis. 
### The variable lists for the manuscript are hard coded below.

#mrnaData = pd.read_csv("breastCancer_mrnaData.csv",index_col=0)
#mrnaData = mrnaData.drop('Severity',axis=1)
#mrnaVar = mrnaData.var()
#mrnaVarGenes = list(mrnaVar.sort_values(ascending=False).index[0:20])

#methData = pd.read_csv("breastCancer_methData.csv",index_col=0)
#methData = methData.drop('Severity',axis=1)
#methVar = methData.var()
#methVarGenes = list(methVar.sort_values(ascending=False).index[0:20])

#protData = pd.read_csv("breastCancer_protData.csv",index_col=0)
#protData = protData.drop('Severity',axis=1)
#protVar = protData.var()
#protVarGenes = list(protVar.sort_values(ascending=False).index[0:20])

#update to include RF Feature Importance list generation
#note: RF Feature Importance is not included, as it varies slighly between independent models 
#note: Two genelist derived from the RF feature importance on the entire dataset are listed below

### Define the Gene Lists

In [ ]:
#Variance gene Lists
mrnaVarGenes = ['CLEC3A', 'CPB1', 'SCGB2A2', 'CSN3', 'S100A7', 'MAGEA6', 'GSTM1',
        'PRAME', 'KCNJ3', 'CYP2B7P1', 'SERPINA6', 'PIP', 'GABRP', 'MSLN',
        'TFF1', 'DHRS2', 'C4orf7', 'SCGB1D2', 'MAGEA3', 'HORMAD1']

methVarGenes = ['OR2M7', 'OR4L1', 'OR5AK2', 'SAMSN1', 'MIR320C1', 'MIR563', 'SLC5A12',
        'OR5B3', 'OR1J4', 'CTSW', 'MNDA', 'OR2T29', 'OR5T2', 'LOC100130331',
        'FLJ41856', 'OR10AG1', 'NCRNA00158', 'OR5M11', 'KRTAP21-2', 'OR6N2']

protVarGenes = ['ESR1', 'PGR', 'MYH11', 'EPPK1', 'GATA3', 'FASN', 'CAV1', 'IGFBP2',
        'KIT', 'ERBB2', 'INPP4B', 'COL6A1', 'HSPA1A', 'GAPDH', 'CCNB1', 'PDCD4',
        'TUBA1B', 'MAPK1','MAPK3', 'AR', 'SCD']

#cell death from gene ontology
cellDeath = ["NDUFA13","BAX","TICAM1","TICAM1","UACA","CASP1",
             "CD3E","FADD","FADD","FADD","FADD","DAPL1","DIDO1",
             "IFI6","RIPK1","RIPK1","CASP2","SENP1","PAWR","TYROBP",
             "PTGIS","CASP3","CD28","BTK","ADORA1",
             "PPARD","IFI27L1","CASP8","IFI27L2",
             "RIPK3","SGPL1","DAP","CRADD","BAK1","CASP8AP2",
             "PRKCA","PDCD6","TFPT","DAPK3","TNFRSF25","FASTK",
             "CFLAR","CD14","TLR4","IFI27","P2RX7","DAP3",
             "CD38","MOAP1","ANXA6","CASP10","ITGAM",
             "TICAM2","DPF2","DAPK1","FAS","FAS",
             "MAP3K5","P2RX4","SHH","CASP12","ST20",
             "CASP5","LY96","CD5","HIP1","FASLG",
             "FASLG","FASLG","TM2D1","CAV1","TLR3"]

cellGrowth = ['tenascin_human','hbo1-5.1_human','hbo1-4.1_human',
              'TMC8','IL17RB','CLSTN3','TFRC','SPOCK1',
              'IGFBP5','KAZALD1','CAMK2D','DDR1','ATAD3A',
              'ARHGEF11','PAPPA2','PRKCQ','MTOR','LAMTOR1',
              'IGFBP4','EGLN2','AGTR1','FAM107A','IGFBP7',
              'KIAA1109','CRKL','IGFBP3','MAD2L2','RAB33B',
              'ITCH','PTCH2','LAMTOR2','FBLN5','NRG3','MUC12',
              'CDK11A','SOCS2','CDC73','PAK5','PLCE1','MELTF',
              'RASGRP2','CISH','WFDC1','CEACAM1','ROS1',
              'EPB41L3','BCAR1','NET1','LTBP4','BAP1','SGK2',
              'AGT','NANOS1','NUBP1','IGFBPL1','RPTOR','SGK3','CLSTN1',
              'PAK4','KIF14','TMEM97','TSG101','CHPT1',
              'EBAG9','CDK11B','SGK1','LMX1A','STK11','URI1','OGFR','TAOK2']

#highest feature imporance from whole dataset Random Forest Analysis
rf1 =  ['CTNND1', 'FAM173B', 'MCOLN2', 'PGK1', 'STAG3L1',
       'ARHGDIB', 'PAK1', 'LOC647288', 'S1PR3', 'FBXO10',
       'C2orf77', 'PSPC1', 'STK19', 'C2', 'GPR84',
       'PWWP2B', 'CYP2S1', 'THTPA', 'CSF2RB', 'AP3S1',
       'COG4', 'STAG3L1', 'FUT4', 'PBLD', 'RPL9',
       'C6orf27', 'HIPK2', 'RPGR', 'CD300LF', 'ECT2',
       'ATG4A', 'GRAMD3', 'SCAP', 'RASGRP1', 'FSIP1',
       'HEMK1', 'HIST1H3E', 'PDZK1', 'CXCL13', 'CHPT1',
       'DOCK10', 'ANAPC2', 'MRPS18C', 'FMNL1', 'IRF2',
       'QPRT', 'KIAA1683', 'GOLGA8A', 'RPL30', 'FCGRT']

rf2 = ['NAGLU','SIGLEC1','RAPGEFL1','NAIP','TMEM156','LYPLA2P1',
        'MAK','PTPN9','NFAM1','MAPK3','CHPT1','CLUL1','PSME2','C2',
        'ZCCHC7','AARS','ZNF382','BCL2A1','PRDM11','GTF2B',
        'MORN1','MDH1B','DHFRL1','CAT','SERPINA3','FZD10','MTMR7',
        'COX7B','LOC100233209','GLIPR2','SLC17A9','IFITM1',
        'ZNF688','NEAT1','TMEM194B','C1orf175','ATP7A','VARS',
        'C7orf29','CYBA','SBK1','DUOX1','DOLK','PAK1','C10orf116',
        'YAP1','FAM103A1','PCGF6','OR51E1','DYX1C1']


### Generate the ML models

In [ ]:
# update to include other gene sets
geneSets = {
    'mRNA Var' : mrnaVarGenes,
    'Methyl Var' : methVarGenes,
    'Prot Var' : protVarGenes,
    'multiomic Var' : mrnaVarGenes + methVarGenes + protVarGenes,
    'Cell Growth' : cellGrowth,
    'Cell Death' : cellDeath,
    'Cell Growth and Death' : cellGrowth + cellDeath,
    'RF1_50' : rf1,
    'RF1_25' : rf1[0:25],
    'RF1_10' : rf1[0:10],
    'RF1_5' : rf1[0:5],
    'RF2_50' : rf2,
    'RF2_25' : rf2[0:25],
    'RF2_10' : rf2[0:10],
    'RF2_5' : rf2[0:5]
}

pointMetricThesh = 0.5
allProba = {}
allROC = pd.DataFrame(columns=['Dataset',
                               'Gene List',
                               'Model',
                               'FPR',
                               'TPR',
                               'Thresholds'])

allMetrics = pd.DataFrame(columns=['DataSet',
                                   'Num Features',
                                   'Gene List',
                                   'Model',
                                   'Precision',
                                   'Recall',
                                   'Accuracy',
                                   'F1',
                                   'AUROC'])

for listname in geneSets.keys():
    y, mrnaData, methData, protData, multiData = buildTrainingDataset(geneSets[listname])

    #update to dict
    dataSets = [ multiData,mrnaData,methData,protData]
    dataSetName = ['Multiomics','mRNA','Methylation','Proteomics' ]

    for i in range(len(dataSetName)):
        if len(dataSets[i].columns) > 0:
            prediction_proba, rocPlots = train_models(dataSets[i],y,listname,dataSetName[i])
            prediction_proba.to_csv(dataSetName[i]+"."+listname+".proba.csv",index = False)
            allProba[dataSetName[i]+"."+listname+".proba.csv"] = prediction_proba
            allROC = allROC.append(rocPlots)

            #update to automatically get model names
            modelNames = [ 'LR','KNN','RF','NB' ]
            for j in modelNames:
                predictions = get_predictions(prediction_proba[j], threshold=pointMetricThesh)
                precision = precision_score(prediction_proba['Severity'],predictions)
                recall = recall_score(prediction_proba['Severity'],predictions)
                accuracy= accuracy_score(prediction_proba['Severity'],predictions)
                f1=f1_score(prediction_proba['Severity'],predictions)
                auc = roc_auc_score(prediction_proba['Severity'],prediction_proba[j])
                modelMetrics = pd.DataFrame([{'DataSet':dataSetName[i],
                                              'Num Features':len(dataSets[i].columns),
                                              'Gene List':listname,
                                              'Model':j,
                                              'Precision':precision,
                                              'Recall':recall,
                                              'Accuracy':accuracy,
                                              'F1':f1,
                                              'AUROC':auc
                                             }])
                allMetrics = allMetrics.append(modelMetrics)


### Output the results

In [ ]:
#allMetrics.to_csv("allMetrics.csv",index=False)
#allROC.to_csv("allROC.csv",index=False)
#for probaTable in allProba.keys():
#    allProba[probaTable].to_csv(probaTable,index=False)
allMetrics, allROC, probaTable